<a href="https://colab.research.google.com/github/iamvishu17/GPT2-Text-Generation-with-KerasNLP/blob/main/Copy_of_gpt2_text_generation_with_kerasnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT2 Text Generation with KerasNLP

**Author:** Chen Qian<br>
**Date created:** 2023/04/17<br>
**Last modified:** 2024/04/12<br>
**Description:** Use KerasNLP GPT2 model and `samplers` to do text generation.

In this tutorial, you will learn to use [KerasNLP](https://keras.io/keras_nlp/) to load a
pre-trained Large Language Model (LLM) - [GPT-2 model](https://openai.com/research/better-language-models)
(originally invented by OpenAI), finetune it to a specific text style, and
generate text based on users' input (also known as prompt). You will also learn
how GPT2 adapts quickly to non-English languages, such as Chinese.

##  Before we begin

Colab offers different kinds of runtimes. Make sure to go to **Runtime ->
Change runtime type** and choose the GPU Hardware Accelerator runtime
(which should have >12G host RAM and ~15G GPU RAM) since you will finetune the
GPT-2 model. Running this tutorial on CPU runtime will take hours.

## Install KerasNLP, Choose Backend and Import Dependencies

This examples uses [Keras 3](https://keras.io/keras_3/) to work in any of
`"tensorflow"`, `"jax"` or `"torch"`. Support for Keras 3 is baked into
KerasNLP, simply change the `"KERAS_BACKEND"` environment variable to select
the backend of your choice. We select the JAX backend below.

In [ ]:
!pip install git+https://github.com/keras-team/keras-nlp.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 21.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras
import tensorflow as tf
import time

keras.mixed_precision.set_global_policy("mixed_float16")

## Introduction to Generative Large Language Models (LLMs)

Large language models (LLMs) are a type of machine learning models that are
trained on a large corpus of text data to generate outputs for various natural
language processing (NLP) tasks, such as text generation, question answering,
and machine translation.

Generative LLMs are typically based on deep learning neural networks, such as
the [Transformer architecture](https://arxiv.org/abs/1706.03762) invented by
Google researchers in 2017, and are trained on massive amounts of text data,
often involving billions of words. These models, such as Google [LaMDA](https://blog.google/technology/ai/lamda/)
and [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html),
are trained with a large dataset from various data sources which allows them to
generate output for many tasks. The core of Generative LLMs is predicting the
next word in a sentence, often referred as **Causal LM Pretraining**. In this
way LLMs can generate coherent text based on user prompts. For a more
pedagogical discussion on language models, you can refer to the
[Stanford CS324 LLM class](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

## Introduction to KerasNLP

Large Language Models are complex to build and expensive to train from scratch.
Luckily there are pretrained LLMs available for use right away. [KerasNLP](https://keras.io/keras_nlp/)
provides a large number of pre-trained checkpoints that allow you to experiment
with SOTA models without needing to train them yourself.

KerasNLP is a natural language processing library that supports users through
their entire development cycle. KerasNLP offers both pretrained models and
modularized building blocks, so developers could easily reuse pretrained models
or stack their own LLM.

In a nutshell, for generative LLM, KerasNLP offers:

- Pretrained models with `generate()` method, e.g.,
    `keras_nlp.models.GPT2CausalLM` and `keras_nlp.models.OPTCausalLM`.
- Sampler class that implements generation algorithms such as Top-K, Beam and
    contrastive search. These samplers can be used to generate text with
    custom models.

## Load a pre-trained GPT-2 model and generate some text

KerasNLP provides a number of pre-trained models, such as [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
and [GPT-2](https://openai.com/research/better-language-models). You can see
the list of models available in the [KerasNLP repository](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

It's very easy to load the GPT-2 model as you can see below:

In [ ]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 416kB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 2.09MB/s]
100%|██████████| 446k/446k [00:00<00:00, 1.32MB/s]
100%|██████████| 484/484 [00:00<00:00, 466kB/s]
100%|██████████| 475M/475M [00:09<00:00, 49.9MB/s]


Once the model is loaded, you can use it to generate some text right away. Run
the cells below to give it a try. It's as simple as calling a single function
*generate()*:

In [ ]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
My trip to Yosemite was the most enjoyable of my life. I was there for three days, and I spent my time exploring the park and enjoying the views of the Yosemite Valley. The view from the summit was breathtaking. The view was also beautiful in the fall, and the views from the summit were beautiful. The view was beautiful in the fall, and the views from the summit were beautiful.

I had the chance to take some pictures of the views from the top of Yosemite, which was the best place to go. I had the chance to take some pictures of the views from the top of Yosemite, which was the best place to go. I had the chance to take some pictures of the views from the top of Yosemite, which was the best place to go. I had the chance to photograph Yosemite Valley, which is a beautiful place to visit. It's a beautiful area, but it was also the only time to see it. It was also one of my favorite places to
TOTAL TIME ELAPSED: 8.70s


Try another one:

In [ ]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is now known as The Fizzie's.

The restaurant, which has been closed for several years, will reopen this summer, the Italian outlet announced on its website.
TOTAL TIME ELAPSED: 1.62s


Notice how much faster the second call is. This is because the computational
graph is [XLA compiled](https://www.tensorflow.org/xla) in the 1st run and
re-used in the 2nd behind the scenes.

The quality of the generated text looks OK, but we can improve it via
fine-tuning.

## More on the GPT-2 model from KerasNLP

Next up, we will actually fine-tune the model to update its parameters, but
before we do, let's take a look at the full set of tools we have to for working
with for GPT2.

The code of GPT2 can be found
[here](https://github.com/keras-team/keras-nlp/blob/master/keras_nlp/models/gpt2/).
Conceptually the `GPT2CausalLM` can be hierarchically broken down into several
modules in KerasNLP, all of which have a *from_preset()* function that loads a
pretrained model:

- `keras_nlp.models.GPT2Tokenizer`: The tokenizer used by GPT2 model, which is a
    [byte-pair encoder](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_nlp.models.GPT2CausalLMPreprocessor`: the preprocessor used by GPT2
    causal LM training. It does the tokenization along with other preprocessing
    works such as creating the label and appending the end token.
- `keras_nlp.models.GPT2Backbone`: the GPT2 model, which is a stack of
    `keras_nlp.layers.TransformerDecoder`. This is usually just referred as
    `GPT2`.
- `keras_nlp.models.GPT2CausalLM`: wraps `GPT2Backbone`, it multiplies the
    output of `GPT2Backbone` by embedding matrix to generate logits over
    vocab tokens.

## Finetune on Reddit dataset

Now you have the knowledge of the GPT-2 model from KerasNLP, you can take one
step further to finetune the model so that it generates text in a specific
style, short or long, strict or casual. In this tutorial, we will use reddit
dataset for example.

In [ ]:
import tensorflow_datasets as tfds

race_ds = tfds.load("race", split="train")

Let's take a look inside sample data from the reddit TensorFlow Dataset. There
are two features:

- **__document__**: text of the post.
- **__title__**: the title.

In [ ]:
for items in race_ds:
    print(items['article'].numpy())
    print(items['questions'].numpy())
    print(items['options'].numpy())
    break

b"Answering the Community Needs of Our City\nThe Silver City Council  recognizes that citizens have certain needs.  To better meet your needs, we have made several changes to community facilities  in 2004. This chart shows how we have tried to make your life better.\nTransport Three stations for the suburbs have been added to the western train service.20 new buses for the southern line were purchased in January. 50 percent of city bus-stops have been upgraded . Buses to the eastern suburbs will run every15 minutes.\nCommunication  Broadband cable  is now available  to all parts of the city. All of the new Government buildings are ' smart'-wired for better computer service!\nMedical Facilities The new state-of-the-art Nightingale Hospital was opened in June. To overcome a shortage of trained medical staff  at Dover Hospital, 10 doctors have been employed from overseas.Some facilities at Station Street Hospital have been upgraded.\nEducation  Textbooks will be free to all primary student

In our case, we are performing next word prediction in a language model, so we
only need the 'document' feature.

In [ ]:
train_ds = (
    race_ds.map(lambda items: items['article'])
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

Now you can finetune the model using the familiar *fit()* function. Note that
`preprocessor` will be automatically called inside `fit` method since
`GPT2CausalLM` is a `keras_nlp.models.Task` instance.

This step takes quite a bit of GPU memory and a long time if we were to train
it all the way to a fully trained state. Here we just use part of the dataset
for demo purposes.

In [ ]:
train_ds = train_ds.take(1000)
num_epochs = 1

# 基于线性衰减的动态学习率
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

# 损失函数
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 模型编译
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

586/586 ━━━━━━━━━━━━━━━━━━━━ 285s 427ms/step - accuracy: 0.3506 - loss: 3.3777


In [ ]:
start = time.time()

output = gpt2_lm.generate("I like to play Games", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I like to play Games for fun and to enjoy the game.                                                                                 The Game is an adventure game. It is a game where you play with friends, play with other friends, play with friends
TOTAL TIME ELAPSED: 6.71s


After fine-tuning is finished, you can again generate text using the same
*generate()* function. This time, the text will be closer to Reddit writing
style, and the generated length will be close to our preset length in the
training set.

## Into the Sampling Method

In KerasNLP, we offer a few sampling methods, e.g., contrastive search,
Top-K and beam sampling. By default, our `GPT2CausalLM` uses Top-k search, but
you can choose your own sampling method.

Much like optimizer and activations, there are two ways to specify your custom
sampler:

- Use a string identifier, such as "greedy", you are using the default
configuration via this way.
- Pass a `keras_nlp.samplers.Sampler` instance, you can use custom configuration
via this way.

In [ ]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball.I love it.I like to watch the game and I love to be involved in the game.
My father and I have been friends for a long time.My mother has always been a big supporter of me.I think my dad was the first to say that basketball is a sport,not a religion.He said, "We're going to be a team,not a team.We're going to be a basketball team."
I remember his first game.He had a great shot and he had to go down for the game.He went down hard and he got the layup.I was so excited.My mom and I had a

GPT-2 output:
I like basketball. I like to play it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I li

For more details on KerasNLP `Sampler` class, you can check the code
[here](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/samplers).

## Finetune on Chinese Poem Dataset

We can also finetune GPT2 on non-English datasets. For readers knowing Chinese,
this part illustrates how to fine-tune GPT2 on Chinese poem dataset to teach our
model to become a poet!

Because GPT2 uses byte-pair encoder, and the original pretraining dataset
contains some Chinese characters, we can use the original vocab to finetune on
Chinese dataset.

In [ ]:
!# Load chinese poetry dataset.
!git clone https://github.com/chinese-poetry/chinese-poetry.git

Cloning into 'chinese-poetry'...
remote: Enumerating objects: 7315, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7315 (delta 2), reused 6 (delta 2), pack-reused 7309
Receiving objects: 100% (7315/7315), 236.97 MiB | 22.55 MiB/s, done.
Resolving deltas: 100% (5000/5000), done.
Updating files: 100% (2285/2285), done.


Load text from the json file. We only use《全唐诗》for demo purposes.

In [ ]:
import os
import json

poem_collection = []
for file in os.listdir("chinese-poetry/全唐诗"):
    if ".json" not in file or "poet" not in file:
        continue
    full_filename = "%s/%s" % ("chinese-poetry/全唐诗", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        poem_collection.extend(content)

paragraphs = ["".join(data["paragraphs"]) for data in poem_collection]

Let's take a look at sample data.

In [ ]:
print(paragraphs[0])

摇落秋風急，蕭條易水寒。悲歌誰激烈，宵雅自平寬。本道尊中國，何庸伐可汗。山居聞備塞，莫厭腐儒餐。


Similar as Reddit example, we convert to TF dataset, and only use partial data
to train.

In [ ]:
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Running through the whole dataset takes long, only take `500` and run 1
# epochs for demo purposes.
train_ds = train_ds.take(500)
num_epochs = 1

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

500/500 ━━━━━━━━━━━━━━━━━━━━ 125s 194ms/step - accuracy: 0.2421 - loss: 2.8193


Let's check the result!

In [ ]:
output = gpt2_lm.generate("昨夜雨疏风骤", max_length=200)
print(output)

昨夜雨疏风骤，白馬清馬深游。白日陰石秋樹，江倚深塵倚風。


In [ ]:
import json # Import the json module

with open('/content/game_of_thrones.json', 'r', encoding='utf-8') as f:
    game_of_thrones_data = json.load(f) # Now you can use json.load()

In [ ]:
# Create a TensorFlow dataset from the JSON data
texts = game_of_thrones_data["texts"]
game_of_thrones_ds = tf.data.Dataset.from_tensor_slices(texts)

In [ ]:
# Prepare the dataset for training
train_ds = (
    game_of_thrones_ds
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
# Take a subset for training (optional, depending on your needs)
train_ds = train_ds.take(1000)
num_epochs = 1

# Configure a dynamically decreasing learning rate based on linear decay
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

In [ ]:
# Define the loss function
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Assuming gpt2_lm is your GPT-2 model
# If not already defined, you need to load or create your GPT-2 model here

# Model compilation
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

# Train the model
gpt2_lm.fit(train_ds, epochs=num_epochs)

679/679 ━━━━━━━━━━━━━━━━━━━━ 259s 351ms/step - accuracy: 0.2995 - loss: 0.6084


In [ ]:
# Inference
start = time.time()
output = gpt2_lm.generate("The Mother of Dragons", max_length=200)
print("\nGPT-2 output:")
print(output)
end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
The Mother of Dragons,  she shouted,  you have a son,  but you will have to wait for him, for the time is up.
TOTAL TIME ELAPSED: 7.19s


Not bad 😀